In [1]:
import yfinance as yf
from datetime import datetime
import quandl
import stockquotes
import numpy as np
import scipy.stats as st 
import pandas as pd 
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [2]:
def Spot(ticker):
    '''
    Takes a ticker symbol and returns the last price of the asset.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    Returns:
    S -- Spot price of the asset
    '''
    
    security = stockquotes.Stock(ticker)
    S = security.current_price

    return S    

In [3]:
S = Spot("^GSPC")
print(f'S&P500 last price was ${S}, thus we will look for strike price in the range of ${round(S-300)} - ${round(S+300)}')

S&P500 last price was $3647.49, thus we will look for strike price in the range of $3347 - $3947


In [4]:
def call_data(ticker,expiration):
    '''
    Extracts the strike price and call premium of an option given the ticker symbol of the underlying asset and 
    the expiration date. Returns a list of strike price and call premium.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    data -- list containig pairs of strike price and call premiums available for the underlying in the expiration selected
    '''
    
    security = yf.Ticker(ticker)
    
    if expiration not in security.options:                       #Assertion to verify that expiration exists
        print('Option expiration not available, please verify')
        return
        
    options = security.option_chain(expiration)
    calls = options.calls
    strike_price = calls['strike']
    call_premium = calls['lastPrice']
    
    data = list(zip(strike_price, call_premium))
    data = np.array(data)

    
    return data

In [5]:
call_data('^GSPC','2021-12-16')

array([[ 900.  , 2755.29],
       [1000.  , 2656.54],
       [1225.  , 1956.5 ],
       [1300.  , 2039.26],
       [1375.  , 2143.78],
       [1400.  , 2119.6 ],
       [1500.  , 2160.15],
       [1525.  , 1425.66],
       [1550.  , 1201.5 ],
       [1600.  , 1600.8 ],
       [1625.  , 1375.  ],
       [1650.  , 1054.5 ],
       [1800.  , 1243.1 ],
       [1825.  , 1466.  ],
       [1900.  , 1361.4 ],
       [2000.  , 1638.  ],
       [2175.  ,  779.5 ],
       [2200.  , 1166.64],
       [2250.  , 1311.3 ],
       [2300.  , 1321.8 ],
       [2350.  ,  834.  ],
       [2400.  , 1007.  ],
       [2425.  ,  760.89],
       [2475.  , 1074.55],
       [2500.  , 1205.23],
       [2525.  ,  575.4 ],
       [2550.  ,  877.68],
       [2575.  ,  895.15],
       [2600.  , 1043.62],
       [2625.  ,  920.  ],
       [2650.  ,  909.5 ],
       [2675.  ,  564.9 ],
       [2700.  ,  960.  ],
       [2725.  ,  756.5 ],
       [2750.  ,  966.06],
       [2775.  ,  371.06],
       [2800.  ,  945.42],
 

In [6]:
def Risk_free(ttm):
    '''
    Extracts the most recent US treasury rate given the time to maturity.
    
    Arguments:
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    Returns:
    rf -- Most recent risk free rate for the time to maturity selected
    '''
    yield_curve = quandl.get('USTREASURY/YIELD', authtoken='JMxryiBcRV26o9r5q7uv')
    rf = yield_curve[ttm][-1]/100
    
    return rf    

In [7]:
Risk_free('1 YR')

0.001

In [8]:
def Time_to_maturity(expiration):
    '''
    Calculates the time to maturity in years from today to the option expiration.
    
    Arguments:
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    ttm -- Time to maturity in year in float format
    '''
    
    today = datetime.now()
    expiration_d = datetime.strptime(expiration,'%Y-%m-%d')
    datetime_delta = (expiration_d - today).days
    ttm = datetime_delta/365
    
    return ttm
    

In [9]:
Time_to_maturity('2021-12-16')

1.0027397260273974

In [10]:
def Norm_d1(ticker, expiration, ttm, sigma = 0.1):
    
    '''
    Calculates the expected value of the stock if the option is exercised using risk-adjusted probabilities.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    N_d1 -- Array of probability to receive the stock given that the stock price is higher than the strike price at 
    expiration
    
    cache -- cache of values needed for Norm_d2() function

    '''
    
    S = Spot(ticker)
    
    data = call_data(ticker,expiration)
    
    K, call_premium = list(zip(*data))
    call_premium = np.array(call_premium)
    
    S = np.array(S)
    K = np.array(K)
    
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    
    d1 = (np.log(np.divide(S,K))+(r+((np.power(sigma,2)/2))*T))/(sigma*np.sqrt(T))
    N_d1 = st.norm.cdf(d1)
    
    cache = (d1, T, r, S, K, sigma, call_premium)
    
    return N_d1, cache

In [11]:
Norm_d1('^GSPC','2021-12-16','1 YR')

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.99999991, 0.99999984, 0.99999948, 0.99999846,
        0.99999571, 0.99998883, 0.99998237, 0.99995801, 0.99993654,
        0.9999054 , 0.9998608 , 0.9997978 , 0.9997099 , 0.9995888 ,
        0.99942396, 0.99920221, 0.99890734, 0.99851959, 0.99801526,
        0.99736625, 0.99653967, 0.99549758, 0.9941967 , 0.99061846,
        0.98822785, 0.98535252, 0.97787173, 0.95392877, 0.91421085,
        0.8553945 , 0.77689017, 0.68162704, 0.57570969, 0.27146008,
        0.1945543 ]),
 (array([14.03492197, 12.98275715, 10.95612302, 10.36270117,  9.80257328,
          9.62263454,  8.933649  ,  8.76858194,  8.60619903,  8.28914606,
          8.13431615,  7.98185014,  7.11292587,  6.97518121,  6.57299278,
          6.06076105,  5.22309291,  5.10896219,  4.88454085,  4.66505224,
  

In [12]:
def Norm_d2(ticker, expiration, ttm, sigma = 0.1):
    '''
    Calculates the probability that the option will be exercised based on risk-adjusted probabilities.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    N_d2 -- Array of probabilities that the option will end up in-the-money thus excercised for each strike price
    
    cache -- Cache of values needed for Call_price() function

    '''
    
    N_d1, cache = Norm_d1(ticker, expiration, ttm, sigma)
    
    d1, T, r, S, K, sigma, call_premium = cache
    
    d2 = np.subtract(d1,sigma*np.sqrt(T))
    N_d2 = st.norm.cdf(d2)
    
    cache = (N_d1, S, K, r, T, call_premium)
    
    return N_d2, cache

In [13]:
Norm_d2('^GSPC','2021-12-16','1 YR')

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.99999985, 0.99999973, 0.99999914, 0.9999975 ,
        0.9999932 , 0.99998263, 0.99997286, 0.9999366 , 0.99990511,
        0.99985986, 0.99979571, 0.99970596, 0.99958196, 0.99941276,
        0.99918462, 0.9988806 , 0.99848008, 0.99795826, 0.99728574,
        0.9964281 , 0.99534562, 0.99399301, 0.99231936, 0.98777769,
        0.98478117, 0.98120774, 0.97203085, 0.94341065, 0.89742469,
        0.8314012 , 0.74588479, 0.64509695, 0.53617141, 0.239305  ,
        0.16818259]),
 (array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 0.99999991, 0.99999984, 0.99999948, 0.99999846,
         0.99999571, 0

In [14]:
def Call_price(ticker, expiration, ttm, sigma = 0.1):
    
    '''
    Call culates the call price of an option from an underlying asset that doesn't pay dividends using the Black-Scholes
    model
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    C -- Array of calculated call price using the Black-Scholes formula for each strike price
    
    call_premium -- Array of market price of a call option for each strike price

    '''
    
    N_d2, cache = Norm_d2(ticker, expiration, ttm, sigma)
    
    N_d1, S, K, r, T, call_premium = cache
    
    C = np.subtract(np.multiply(S,N_d1),np.multiply(K,np.multiply(np.exp(-r*T),N_d2)))
    
    call_premium = np.array(call_premium)
    
    return C, call_premium

In [15]:
Call_price('^GSPC','2021-12-16','1 YR')

(array([2748.39201344, 2648.49223715, 2423.71774051, 2348.7929083 ,
        2273.86807608, 2248.89313201, 2148.99335573, 2124.01841165,
        2099.04346758, 2049.09357944, 2024.11863537, 1999.1436913 ,
        1849.29402687, 1824.3190828 , 1749.39425059, 1649.49447434,
        1474.66987167, 1449.69493275, 1399.7450703 , 1349.79525923,
        1299.84558686, 1249.89626463, 1224.92186795, 1174.97412411,
        1150.00113229, 1125.02908946, 1100.05840034, 1075.08961698,
        1050.12348231, 1025.16098319, 1000.20341379,  975.25245016,
         950.31023653,  925.37948343,  900.46357725,  875.56670042,
         850.69396058,  825.85152659,  801.04676848,  751.58660342,
         726.95318046,  702.40164321,  653.6074274 ,  557.80374285,
         465.69409326,  379.16865796,  300.2325022 ,  230.65137396,
         171.612785  ,   57.79382469,   37.57872517]),
 array([2755.29, 2656.54, 1956.5 , 2039.26, 2143.78, 2119.6 , 2160.15,
        1425.66, 1201.5 , 1600.8 , 1375.  , 1054.5 , 1243.

In [16]:
def difference(ticker, expiration, ttm, sigma = 0.1):
    '''
    Implements the difference between the call price calculated by the Black-Scholes model and the market call price for
    each strike price
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1

    Returns:
    difference -- Array of differences between Black-Scholes call price and market call price for each strike price
    '''
    
    B_S, call_premium = Call_price(ticker, expiration, ttm, sigma)
    
    difference = np.subtract(B_S, call_premium)
    
    return difference

In [17]:
def vega(ticker, expiration, ttm, sigma=0.1):
    S = Spot(ticker)
    
    T = Time_to_maturity(expiration)
    
    N_d1, cache = Norm_d1(ticker, expiration, ttm, sigma)
    d1, T, r, S, K, sigma, call_premium = cache
    
    
    v = -S*np.sqrt(T)*st.norm.pdf(d1)
    
    return v, sigma
    

In [18]:
vega('^GSPC','2021-12-16','1 YR')

(array([-2.45495971e-40, -3.65510766e-34, -1.25278027e-23, -6.99869146e-21,
        -1.98483197e-18, -1.13953787e-17, -6.80704548e-15, -2.93409966e-14,
        -1.20261623e-13, -1.75115203e-12, -6.24446519e-12, -2.13335139e-11,
        -1.50387867e-08, -3.96826236e-08, -6.05087320e-07, -1.53839126e-05,
        -1.73607965e-03, -3.13062276e-03, -9.60813015e-03, -2.74021933e-02,
        -7.29274109e-02, -1.81820243e-01, -2.80465090e-01, -6.38262403e-01,
        -9.42396662e-01, -1.37238054e+00, -1.97188715e+00, -2.79647602e+00,
        -3.91571651e+00, -5.41535585e+00, -7.39942624e+00, -9.99216080e+00,
        -1.33395659e+01, -1.76104813e+01, -2.29969495e+01, -2.97137206e+01,
        -3.79967264e+01, -4.81003903e+01, -6.02936736e+01, -9.20647675e+01,
        -1.12199456e+02, -1.35520953e+02, -1.92645230e+02, -3.52818675e+02,
        -5.72306387e+02, -8.30957545e+02, -1.09018838e+03, -1.30337502e+03,
        -1.43081131e+03, -1.21093351e+03, -1.00562154e+03]),
 0.1)

In [19]:
def vega(ticker, expiration, ttm, sigma = 0.1):
    S = Spot(ticker)
    print('S =',S)
    
    T = Time_to_maturity(expiration)
    print('T =',T)
    
    data = call_data(ticker, expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    print('K =',K[0])
    
    r = Risk_free(ttm)
    print('r =',r)
    
    A = np.log(S/K) + (r + (sigma**2/2))*T
    print('A =',A)
    
    B = A**2/(2*sigma**2*T)
    print('B =',B)
    
    
    v = S * (np.sqrt(T/(2*math.pi))) * np.exp(-(np.log(S/K) + (r+sigma**2/2)*T)**2/(2*sigma**2*T))
    
    return v

In [20]:
vega('^GSPC','2021-12-16','1 YR')

S = 3647.49
T = 1.0027397260273974
K = 900.0
r = 0.001
A = [ 1.40541621  1.3000557   1.09711485  1.03769143  0.98160197  0.96358346
  0.89459059  0.87806129  0.86180077  0.83005207  0.81454788  0.79928041
  0.71226903  0.69847571  0.65820181  0.60690852  0.52302703  0.51159834
  0.48912548  0.46714658  0.44564037  0.42458696  0.41422417  0.3938153
  0.38376497  0.37381464  0.36396234  0.35420616  0.34454425  0.3349748
  0.32549606  0.31610632  0.30680393  0.29758727  0.28845479  0.27940495
  0.27043628  0.26154733  0.2527367   0.23534496  0.22676122  0.21825053
  0.20144341  0.16865359  0.13690489  0.10613323  0.07628027  0.04729273
  0.01912185 -0.06092085 -0.08623866]
B = [9.84899014e+01 8.42763468e+01 6.00186156e+01 5.36930713e+01
 4.80454896e+01 4.62978110e+01 3.99052866e+01 3.84442546e+01
 3.70335663e+01 3.43551980e+01 3.30837722e+01 3.18551842e+01
 2.52970518e+01 2.43267673e+01 2.16022969e+01 1.83665780e+01
 1.36404927e+01 1.30508871e+01 1.19295033e+01 1.08814839e+01
 9.90263646e

array([2.45401721e-40, 3.65380957e-34, 1.25240479e-23, 6.99670746e-21,
       1.98429971e-18, 1.13923790e-17, 6.80538188e-15, 2.93339583e-14,
       1.20233309e-13, 1.75075493e-12, 6.24307562e-12, 2.13288556e-11,
       1.50358603e-08, 3.96750512e-08, 6.04978513e-07, 1.53813619e-05,
       1.73583157e-03, 3.13018519e-03, 9.60684620e-03, 2.73986960e-02,
       7.29185319e-02, 1.81799152e-01, 2.80433350e-01, 6.38193731e-01,
       9.42297854e-01, 1.37224038e+00, 1.97169107e+00, 2.79620539e+00,
       3.91534791e+00, 5.41486024e+00, 7.39876821e+00, 9.99129784e+00,
       1.33384478e+01, 1.76090495e+01, 2.29951372e+01, 2.97114523e+01,
       3.79939189e+01, 4.80969531e+01, 6.02895102e+01, 9.20588478e+01,
       1.12192505e+02, 1.35512872e+02, 1.92634628e+02, 3.52802418e+02,
       5.72284980e+02, 8.30933450e+02, 1.09016566e+03, 1.30335818e+03,
       1.43080383e+03, 1.21095367e+03, 1.00564524e+03])

In [21]:
import time

In [22]:
def update_sigma(iterations, ticker, expiration, ttm, learning_rate=0.075):
    
    tic = time.time()
    implied_vol = np.ones(51)
    implied_vol = implied_vol*0.8
    
    
    S = Spot(ticker)
    
    data = call_data(ticker,expiration)
    
    K, call_premium = list(zip(*data))
    call_premium = np.array(call_premium)
    
    S = np.array(S)
    K = np.array(K)
    
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)

    for i in range(0,iterations):
        if i%100 == 0:
            print('iteration #',i)
            print('implied vol beg:',implied_vol[0])

        d1 = (np.log(np.divide(S,K))+(r+((np.power(implied_vol,2)/2))*T))/(implied_vol*np.sqrt(T))
        N_d1 = st.norm.cdf(d1)
        
        d2 = np.subtract(d1,implied_vol*np.sqrt(T))
        N_d2 = st.norm.cdf(d2)
        
        C = np.subtract(np.multiply(S,N_d1),np.multiply(K,np.multiply(np.exp(-r*T),N_d2)))
        if i%100 == 0:        
            print('BS premium =',C[0])
            print('Market premium =',call_premium[0])

#         C, call_premium = Call_price(ticker, expiration, ttm, implied_vol)

        difference = np.subtract(C, call_premium)
        if i%100 == 0:
            print('The difference is:',difference[0])

        v = S*np.sqrt(T)*st.norm.pdf(d1)
        if i%100 == 0:
            print('vega =',v[0])
    
#         v, sigma = vega(ticker, expiration, ttm)
        for j in range(0,len(difference)):
            if difference[j] >= 0:
                difference[j] = difference[j]-learning_rate*v[j]
            else:
                difference[j] = difference[j]+learning_rate*v[j]
            
        if i%100 == 0:
            print('implied vol end:',implied_vol[0])
            print('-'*20)
            print('\n')

    toc = time.time()
    
    print(toc-tic)
    return implied_vol
        

In [ ]:
update_sigma(5000,'^GSPC','2021-12-16','1 YR')

__References:__<p>&nbsp;</p> 

- Tim Worall (2008). The Black-Scholes Formula, FIN-40008 Financial Instruments http://www.timworrall.com/fin-40008/bscholes.pdf

In [71]:
def implied_vol(iterations, ticker, expiration, ttm): 
    
    S = Spot(ticker)
    S = np.array(S)
    
    data = call_data(ticker,expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    call_premium = np.array(call_premium)
      
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    sigma_ran = np.random.randn(iterations)
    
    imply_vol = []
    difference = []
    
    for i in range(0,len(K)):

        spread_pos = []
        sigma_pos = []
        spread_neg = []
        sigma_neg = []
        
        for j in range(0,len(sigma_ran)):

            d1 = (np.log(np.divide(S,K[i]))+(r+((np.power(sigma_ran[j],2)/2))*T))/(sigma_ran[j]*np.sqrt(T))
            N_d1 = st.norm.cdf(d1)

            d2 = np.subtract(d1,sigma_ran[j]*np.sqrt(T))
            N_d2 = st.norm.cdf(d2)

            C = np.subtract(np.multiply(S,N_d1),np.multiply(K[i],np.multiply(np.exp(-r*T),N_d2)))

            spread = np.subtract(C, call_premium[i])

            if spread >=0:
                spread_pos.append(spread)
                sigma_pos.append(sigma_ran[j])
            else:
                spread_neg.append(spread)
                sigma_neg.append(sigma_ran[j])

        min_dif = np.min(spread_pos)
        index_min_dif = np.argmin(spread_pos)

        sigma_optimun = sigma_pos[index_min_dif]

        imply_vol.append(sigma_optimun)
        difference.append(min_dif)

    
    print(imply_vol)
    print(difference)
    print(len(imply_vol))
    print(len(difference))
     
    return imply_vol, difference

In [72]:
implied_vol(1000,'^GSPC','2021-12-16','1 YR')

[0.6639016769294191, 0.627853082463662, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.11867744844298468, 0.4684023365594395, 0.10855585369971817, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.09650999617092758, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.04431393742227567, 0.05492948065150353, 0.04431393742227567, 0.050713743529200696, 0.04936628047611569, 0.3198689280348223, 0.04431393742227567, 0.04431393742227567, 0.041167669530286576, 0.041167669530286576, 0.041167669530286576, 0.041167669530286576, 0.041167669530286576, 0.1820088658247267, 0.010215698018473546, 0.2722137657439149, 0.010215698018473546, 0.2993126523855625, 0.010215698018473546, 0.22771016616586412, 0.23206981312059743, 0.010215698018473546, 0.2582174501217897, 0.2553097167114459, 0.2418607036087867, 0.2604247306981538, 0.2553097167114459, 0.24398573597071535, 0.2159023798405

([0.6639016769294191,
  0.627853082463662,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.11867744844298468,
  0.4684023365594395,
  0.10855585369971817,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.09650999617092758,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.04431393742227567,
  0.05492948065150353,
  0.04431393742227567,
  0.050713743529200696,
  0.04936628047611569,
  0.3198689280348223,
  0.04431393742227567,
  0.04431393742227567,
  0.041167669530286576,
  0.041167669530286576,
  0.041167669530286576,
  0.041167669530286576,
  0.041167669530286576,
  0.1820088658247267,
  0.010215698018473546,
  0.2722137657439149,
  0.010215698018473546,
  0.2993126523855625,
  0.010215698018473546,
  0.22771016616586412,
  0.23206981312059743,
  0.010215698018473546,
  0.2582174501217897,
  0.2553097167114459,
  0.2418607

In [83]:
def implied_vol2(iterations, ticker, expiration, ttm):  
    
    S = Spot(ticker)
    S = np.array(S)
    
    data = call_data(ticker,expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    call_premium = np.array(call_premium)
      
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    sigma_ran = np.random.randn(iterations)
    
    imply_vol = []
    difference = []
    
    for i in range(0,len(K)):

        spread_pos = []
        sigma_pos = []
        spread_neg = []
        sigma_neg = []

        for j in range(0,len(sigma_ran)):

            d1 = (np.log(np.divide(S,K[i]))+(r+((np.power(sigma_ran[j],2)/2))*T))/(sigma_ran[j]*np.sqrt(T))
            N_d1 = st.norm.cdf(d1)

            d2 = np.subtract(d1,sigma_ran[j]*np.sqrt(T))
            N_d2 = st.norm.cdf(d2)

            C = np.subtract(np.multiply(S,N_d1),np.multiply(K[i],np.multiply(np.exp(-r*T),N_d2)))

            spread = np.subtract(C, call_premium[i])

            if spread >=0:
                spread_pos.append(spread)
                sigma_pos.append(sigma_ran[j])
            else:
                spread_neg.append(spread)
                sigma_neg.append(sigma_ran[j])

        min_dif = np.min(spread_pos)
        index_min_dif = np.argmin(spread_pos)

        max_dif = np.max(spread_neg)
        index_max_dif = np.argmax(spread_neg)

        if abs(min_dif) < abs(max_dif):
            sigma_optimun = sigma_pos[index_min_dif]
            spr = min_dif
        else:
            sigma_optimun = sigma_neg[index_max_dif]
            spr = max_dif

        imply_vol.append(sigma_optimun)
        difference.append(spr)

    
    print(imply_vol)
    print(difference)
    print(len(imply_vol))
    print(len(difference))
     
#     return imply_vol, difference

In [ ]:
implied_vol2(1000,'^GSPC','2021-12-16','1 YR')